<a href="https://colab.research.google.com/github/reesha-rsh/MLb4/blob/main/Homework/HW9_Boosting_XGBoost%2C_LightGBM%2C_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework:

- use the dataset and validation approach which you used for previous homeworks
- train xgboost, lightgbm, and catboost models and find the best hyperparameters for each algorithm
- compare results between them and previously trained random forest

# Init

In [1]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

train_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/train.csv")
test_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/test.csv")

Mounted at /content/drive


In [2]:
random_state = 42

In [3]:
def generate(df,age_median,fare_median):
  useless_features = ['Name','Ticket','Cabin']
  data_cleaned = df
  data_cleaned = data_cleaned.drop(columns = useless_features)

  # generate binary values using get_dummies
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Sex'],prefix=["Sex"])
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Embarked'],prefix=["Embarked"])

  # Check for NaN values in the DataFrame
  nan_mask = data_cleaned.isnull()
  # Count the number of NaN values in each column
  nan_count_per_column = data_cleaned.isnull().sum()

  data_cleaned['Age'] = data_cleaned['Age'].fillna(age_median)
  data_cleaned['Fare'] = data_cleaned['Fare'].fillna(fare_median)

  return data_cleaned


In [4]:
not_features = ['PassengerId', 'Survived']

In [5]:
# get medians that will fill NaNs in generate func
age_median = train_full['Age'].median()
print(age_median)
fare_median = train_full['Fare'].median()
print(fare_median)

28.0
14.4542


In [6]:
train = generate(train_full,age_median=age_median,fare_median=fare_median)
print(train)

X_train = train.drop(not_features, axis = 1)
y_train = train['Survived']

     PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  Sex_female  \
0              1         0       3  22.0      1      0   7.2500           0   
1              2         1       1  38.0      1      0  71.2833           1   
2              3         1       3  26.0      0      0   7.9250           1   
3              4         1       1  35.0      1      0  53.1000           1   
4              5         0       3  35.0      0      0   8.0500           0   
..           ...       ...     ...   ...    ...    ...      ...         ...   
886          887         0       2  27.0      0      0  13.0000           0   
887          888         1       1  19.0      0      0  30.0000           1   
888          889         0       3  28.0      1      2  23.4500           1   
889          890         1       1  26.0      0      0  30.0000           0   
890          891         0       3  32.0      0      0   7.7500           0   

     Sex_male  Embarked_C  Embarked_Q  Embarked_S  

In [7]:
test = generate(test_full,age_median=age_median,fare_median=fare_median)
X_test = test.drop('PassengerId' , axis = 1)

In [8]:
# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [9]:
# Define the custom scoring function with the desired beta value
beta = 0.5
custom_scorer = make_scorer(fbeta_score, beta=beta)

#XGBoost

In [10]:
import xgboost as xgb

In [11]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

num_rounds = 10000
param_grid = {
    #default
    "objective": "binary:logistic",
    "eta": 0.05,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": random_state,
    "eval_metric": "auc",

    # regularization parameters
    "max_leaves": 0,
    "subsample": 0.8,
    "colsample_bytree": 0.7,
    "gamma": 0,
    "min_child_weight": 5,


    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(param_grid, xgb_train, num_rounds, early_stopping_rounds=50,folds=skf, verbose_eval=20)

[0]	train-auc:0.85474+0.00676	test-auc:0.83633+0.01643
[20]	train-auc:0.89575+0.00375	test-auc:0.86894+0.01969
[40]	train-auc:0.90691+0.00441	test-auc:0.86925+0.02466
[60]	train-auc:0.91338+0.00460	test-auc:0.87305+0.02571
[80]	train-auc:0.91909+0.00367	test-auc:0.87604+0.02383
[100]	train-auc:0.92372+0.00372	test-auc:0.87534+0.02377
[120]	train-auc:0.92830+0.00398	test-auc:0.87579+0.02487
[140]	train-auc:0.93138+0.00371	test-auc:0.87670+0.02375
[160]	train-auc:0.93512+0.00386	test-auc:0.87772+0.02366
[180]	train-auc:0.93838+0.00365	test-auc:0.87996+0.02387
[200]	train-auc:0.94117+0.00347	test-auc:0.87975+0.02396
[220]	train-auc:0.94386+0.00351	test-auc:0.88025+0.02388
[240]	train-auc:0.94594+0.00327	test-auc:0.88204+0.02462
[260]	train-auc:0.94815+0.00340	test-auc:0.88186+0.02570
[280]	train-auc:0.95010+0.00323	test-auc:0.88168+0.02454
[300]	train-auc:0.95202+0.00292	test-auc:0.88257+0.02592
[320]	train-auc:0.95329+0.00298	test-auc:0.88269+0.02445
[340]	train-auc:0.95476+0.00301	test-

In [12]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

num_rounds = 10000
param_grid = {
    #default
    "objective": "binary:logistic",
    "eta": 0.05,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": random_state,
    "eval_metric": "auc",

    # regularization parameters
    "max_leaves": 0,
    "subsample": 0.8,
    "colsample_bytree": 0.7,
    "gamma": 0,
    "min_child_weight": 5,


    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(param_grid, xgb_train, num_rounds, early_stopping_rounds=50,folds=skf, verbose_eval=20)

[0]	train-auc:0.85474+0.00676	test-auc:0.83633+0.01643
[20]	train-auc:0.89575+0.00375	test-auc:0.86894+0.01969
[40]	train-auc:0.90691+0.00441	test-auc:0.86925+0.02466
[60]	train-auc:0.91338+0.00460	test-auc:0.87305+0.02571
[80]	train-auc:0.91909+0.00367	test-auc:0.87604+0.02383
[100]	train-auc:0.92372+0.00372	test-auc:0.87534+0.02377
[120]	train-auc:0.92830+0.00398	test-auc:0.87579+0.02487
[140]	train-auc:0.93138+0.00371	test-auc:0.87670+0.02375
[160]	train-auc:0.93512+0.00386	test-auc:0.87772+0.02366
[180]	train-auc:0.93838+0.00365	test-auc:0.87996+0.02387
[200]	train-auc:0.94117+0.00347	test-auc:0.87975+0.02396
[220]	train-auc:0.94386+0.00351	test-auc:0.88025+0.02388
[240]	train-auc:0.94594+0.00327	test-auc:0.88204+0.02462
[260]	train-auc:0.94815+0.00340	test-auc:0.88186+0.02570
[280]	train-auc:0.95010+0.00323	test-auc:0.88168+0.02454
[300]	train-auc:0.95202+0.00292	test-auc:0.88257+0.02592
[320]	train-auc:0.95329+0.00298	test-auc:0.88269+0.02445
[340]	train-auc:0.95476+0.00301	test-

In [13]:
param_grid = {
    #default
    "objective": ["binary:logistic"],
    "eta": [0.01, 0.05],
    "verbosity": [0],
    "nthread": [10],
    "random_seed": [random_state],


    # regularization parameters
    "max_leaves": [0, 16],
    "subsample": [0.7, 0.8, 0.9 ],
    "colsample_bytree": [0.7, 0.8, 0.9 ],
    "min_child_weight": [1, 5, 10],

    #lightgbm approach
    "tree_method": ["hist"],
    "grow_policy": ["lossguide"],

    "n_estimators" : [700]
}

In [14]:

xgb_model = xgb.XGBClassifier()
grid_search = GridSearchCV(xgb_model, param_grid, scoring=custom_scorer, cv=skf )

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
# Print the best hyperparameters
print("Best Hyperparameters:", best_params)


fbeta = cross_val_score(best_model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(best_model, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

Best Hyperparameters: {'colsample_bytree': 0.7, 'eta': 0.01, 'grow_policy': 'lossguide', 'max_leaves': 16, 'min_child_weight': 5, 'n_estimators': 700, 'nthread': 10, 'objective': 'binary:logistic', 'random_seed': 42, 'subsample': 0.7, 'tree_method': 'hist', 'verbosity': 0}
F-beta Score (beta=0.5): 0.8163
Accuracy: 0.8406


In [15]:
y_pred = best_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionXgb.csv", index=False)

**SUBMISSION SCORE:** 0.77751

# LightGBM

In [10]:
import lightgbm as lgb

In [51]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.1,
    "num_threads": 10,
    "metric": "auc",
    "seed":  [random_state],

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "num_leaves": 25,
    "min_child_weight": 1,
    "lambda_l1": 2,
    "lambda_l2": 1

}


n_rounds = 10000

lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, callbacks=([lgb.early_stopping(stopping_rounds=20)]), eval_train_metric=True)

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001459 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.00

In [11]:
param_grid = {
    #default
    "objective": ["binary"],
    "learning_rate": [0.1],
    "num_threads": [10],

    "seed":  [random_state],

    #regularization
    "colsample_bytree": [0.7, 0.8, 0.9 ],
    "subsample": [0.7, 0.8, 0.9 ],
    "subsample_freq": [1],
    "num_leaves": [None, 25],
    "min_child_weight": [1, 5, 10],

    "n_estimators": [700],

    "lambda_l1": [2],
    "lambda_l2": [1]

}

lgb_model = lgb.LGBMClassifier()

grid_search = GridSearchCV(lgb_model, param_grid, scoring=custom_scorer, cv=skf )

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
# Print the best hyperparameters



fbeta = cross_val_score(best_model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(best_model, X_train, y_train, scoring='accuracy', cv=skf).mean()



Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [12]:
print("Best Hyperparameters:", best_params)
print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

Best Hyperparameters: {'colsample_bytree': 0.7, 'lambda_l1': 2, 'lambda_l2': 1, 'learning_rate': 0.1, 'min_child_weight': 5, 'n_estimators': 700, 'num_leaves': None, 'num_threads': 10, 'objective': 'binary', 'seed': 42, 'subsample': 0.8, 'subsample_freq': 1}
F-beta Score (beta=0.5): 0.8224
Accuracy: 0.8485


In [63]:
y_pred = best_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionLgb.csv", index=False)

**SUBMISSION SCORE:** 0.76067

# CatBoost

In [64]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.5 MB/s eta 0:00:00


In [74]:
categorical_features_indices = np.where(X_train.dtypes != float)[0]

In [86]:
import catboost as ctb
parameters = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 700,
    "learning_rate": 0.01,
    "random_seed": random_state,
    "od_wait": 20,
    "od_type": "Iter",
    "thread_count": 10,
    "max_depth": None
}

ctb_data = ctb.Pool(X_train,y_train,cat_features=categorical_features_indices)
result = ctb.cv(ctb_data, parameters, folds=skf, seed=random_state, verbose_eval=20, plot=False)

Training on fold [0/5]
0:	test: 0.8657444	best: 0.8657444 (0)	total: 8.04ms	remaining: 5.62s
20:	test: 0.8663373	best: 0.8663373 (20)	total: 151ms	remaining: 4.89s
40:	test: 0.8737813	best: 0.8751647 (37)	total: 296ms	remaining: 4.76s

bestTest = 0.8751646904
bestIteration = 37

Training on fold [1/5]
0:	test: 0.8414439	best: 0.8414439 (0)	total: 7.27ms	remaining: 5.08s
20:	test: 0.8633021	best: 0.8654412 (16)	total: 174ms	remaining: 5.62s
40:	test: 0.8653743	best: 0.8667781 (22)	total: 321ms	remaining: 5.16s

bestTest = 0.8667780749
bestIteration = 22

Training on fold [2/5]
0:	test: 0.8042781	best: 0.8042781 (0)	total: 12.4ms	remaining: 8.7s
20:	test: 0.8292781	best: 0.8424465 (3)	total: 173ms	remaining: 5.6s

bestTest = 0.8424465241
bestIteration = 3

Training on fold [3/5]
0:	test: 0.8333556	best: 0.8333556 (0)	total: 13.6ms	remaining: 9.48s
20:	test: 0.8312166	best: 0.8342246 (17)	total: 261ms	remaining: 8.43s
40:	test: 0.8395722	best: 0.8399733 (38)	total: 526ms	remaining: 8.46s


In [88]:
param_grid = {
    "loss_function": ["Logloss"],
    "eval_metric": ["AUC"],
    "iterations": [700],
    "learning_rate": [0.01],
    "random_seed": [random_state],
    "od_wait": [20],
    "od_type": ["Iter"],
    "thread_count": [10],
    "subsample": [0.6, 0.7, 0.8, 0.9],
    "colsample_bylevel": [0.6, 0.7, 0.8, 0.9],
    "min_data_in_leaf": [1]

}

clf = ctb.CatBoostClassifier()

grid_search = GridSearchCV(clf, param_grid, scoring=custom_scorer, cv=skf )

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
# Print the best hyperparameters
print("Best Hyperparameters:", best_params)


fbeta = cross_val_score(best_model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(best_model, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

Streaming output truncated to the last 5000 lines.
602:	total: 1.28s	remaining: 206ms
603:	total: 1.28s	remaining: 204ms
604:	total: 1.28s	remaining: 202ms
605:	total: 1.29s	remaining: 200ms
606:	total: 1.29s	remaining: 197ms
607:	total: 1.29s	remaining: 195ms
608:	total: 1.29s	remaining: 193ms
609:	total: 1.29s	remaining: 191ms
610:	total: 1.29s	remaining: 188ms
611:	total: 1.29s	remaining: 186ms
612:	total: 1.3s	remaining: 184ms
613:	total: 1.3s	remaining: 182ms
614:	total: 1.3s	remaining: 180ms
615:	total: 1.3s	remaining: 178ms
616:	total: 1.31s	remaining: 176ms
617:	total: 1.31s	remaining: 174ms
618:	total: 1.31s	remaining: 172ms
619:	total: 1.31s	remaining: 170ms
620:	total: 1.32s	remaining: 168ms
621:	total: 1.32s	remaining: 166ms
622:	total: 1.32s	remaining: 164ms
623:	total: 1.33s	remaining: 162ms
624:	total: 1.33s	remaining: 160ms
625:	total: 1.33s	remaining: 157ms
626:	total: 1.33s	remaining: 155ms
627:	total: 1.33s	remaining: 153ms
628:	total: 1.34s	remaining: 151ms
629:	tot

In [89]:
y_pred = best_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionCat.csv", index=False)

**SUBMISSION SCORE:** 0.78468

#Conclusion


- **Random forest**
  - Submission Accuracy: 0.76555
  - F-beta Score (beta=0.5): 0.8119
  - Accuracy: 0.8440
- **XGBoost**
  - Submission Accuracy: 0.77751
  - F-beta Score (beta=0.5): 0.8163
  - Accuracy: 0.8406
- **LightGBM**
  - Submission Accuracy: 0.76067
  - F-beta Score (beta=0.5): 0.8224
  - Accuracy: 0.8485
- **CatBoost**:
  - Submission Accuracy: 0.78468
  - F-beta Score (beta=0.5): 0.8164
  - Accuracy: 0.8384

Generally boosting models perform better. For LightGBM I have got the highest F bwgta score but Accuracy on submission was lower than Random Forest. CatBoost performs quite well. Means it's way of working with categories seems to work.